# ஒரு விலங்கு நிபுணர் அமைப்பை செயல்படுத்துதல்

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) இல் இருந்து ஒரு உதாரணம்.

இந்த எடுத்துப்பாட்டில், சில உடல் பண்புகளை அடிப்படையாகக் கொண்டு ஒரு விலங்கை தீர்மானிக்க ஒரு எளிய அறிவு அடிப்படையிலான அமைப்பை செயல்படுத்துவோம். இந்த அமைப்பை பின்வரும் AND-OR மரமாக பிரதிபலிக்கலாம் (இது முழுமையான மரத்தின் ஒரு பகுதி, நாங்கள் எளிதில் சில கூடுதல் விதிகளையும் சேர்க்கலாம்):

![](../../../../../../translated_images/ta/AND-OR-Tree.5592d2c70187f283.webp)


## நமதுயான நிபுணத்துவ அமைப்புகளுக்கான ஷெல் பின்வாங்கி தீர்வுடன்

உற்பத்தி விதிகளின் அடிப்படையில் அறிவு பிரதிநிதித்துவத்திற்கு எளிய மொழியை வரையறுக்க முயலலாம். விதிகளை வரையறுக்க பைத்தான் வகுப்புகளை முக்கிய வார்த்தைகளாக பயன்படுத்துவோம். மூன்று வகையான வகுப்புகள் இருக்கின்றன:
* `Ask` என்பது பயனரிடம் கேட்க வேண்டிய கேள்வியை பிரதிநிதித்துவம் செய்யும். இது சாத்தியமான பதில்களின் தொகுப்பை கொண்டுள்ளது.
* `If` என்பது ஒரு விதியை பிரதிநிதித்துவம் செய்யும், மற்றும் அது விதியின் உள்ளடக்கத்தை சேமிப்பதற்கான சொல்லியல் சர்க்கரை மாதிரி உள்ளது
* `AND`/`OR` என்பது மரத்தின் AND/OR கிளைகளைக் குறிப்பிடும் வகுப்புகள். அவை உள்ளடக்க வாயிலான கணக்குகளை மட்டும் சேமிக்கின்றன. குறியுறுப்பை எளிமையாக்க, எல்லா செயல்பாடும் பெற்றோர் வகுப்பு `Content`-ல் வரையறுக்கப்பட்டுள்ளது


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

எமது கணினி அமைப்பில், செயல்பாட்டுக் கால நினைவகம் **அறிவியல் தரவுகளின்** பட்டியலை **பண்பு-மதிப்புப் பொருப்புகளாக** கொண்டிருக்கும். அறிவுத்தளம் ஒரே பெரிய அகராதியாக வரையறுக்கப்படலாம், அது செயல்களை (செயற்பாட்டுக் கால நினைவகத்தில் சேர்க்கப்படவேண்டிய புதிய அறிவியல் தரவுகள்) AND-OR வெளிப்பாடுகளாக வெளிப்படுத்தப்பட்ட நிபந்தனைகளுடன் இணைக்கிறது. மேலும், சில அறிவியல் தரவுகளை `Ask` செய்யலாம்.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

பின்னோக்கிய ஊகத்தை செய்ய, நாம் `Knowledgebase` வகுப்பை வரையறுப்போம். இது கொண்டிருக்க இருக்கும்:
* பணியாற்றும் `memory` - பண்புகளை மதிப்புகளுக்கு மையப்படுத்தும் அகராதி
* அறிவுத்தளம் `rules` முன்பு வரையறுக்கப்பட்ட வடிவத்தில்

இரு முக்கிய முறைமைகள்:
* `get` பண்பின் மதிப்பை பெற, தேவையானபட்சத்தில் ஊகத்தைச் செய்கிறது. உதாரணமாக, `get('color')` ஒரு வண்ண வகை மதிப்பை பெறும் (தேவையானால் கேட்கும், மற்றும் பின்னர் பணியாற்றும் நினைவகத்தில் மதிப்பை சேமிக்கும்). நாம் `get('color:blue')` என்றால், அது வண்ணத்திற்காக கேட்கும், பின்னர் வண்ணத்தின் அடிப்படையில் `y`/`n` மதிப்பை தரும்.
* `eval` உண்மையான ஊகத்தை செய்கிறது, அதாவது AND/OR மரத்தை கடந்துமுழுக்குகிறது, துணை-நோக்குகளை மதிப்பாய்வு செய்கிறது, மற்றும் மேலும்.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

இப்போது நம் விலங்கு அறிவுக் கூறுகளை வரையறுக்கலாம் மற்றும் ஆலோசனையை முன்னெடுக்கலாம். இந்த அழைப்பு உங்களுக்குப் பின்வரும் கேள்விகளை கேட்கும் என்பதை கவனியுங்கள். நீங்கள் ஆம்-இல்லை கேள்விகளுக்கு `y`/`n` என்று எழுதுவதால் பதிலளிக்கலாம், அல்லது பல தேர்வுகள் உள்ள கேள்விகளுக்கு எண் (0..N) குறிப்பிடுவதால் பதில் அளிக்கலாம்.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## முன்னோக்கி நுண்ணறிவுக்கு Experta பயன்பாடு

அடுத்த உதாரணத்தில், அறிவு பிரதிநிதித்துவத்திற்கு பயன்படுத்தப்படும் புத்தகங்களிலொன்றான [Experta](https://github.com/nilp0inter/experta) யை பயின்று முன் நோக்கி நுண்ணறிவை செயல்படுத்த முயற்சிப்போம். **Experta** என்பது Python இல் முன் நோக்கி நுண்ணறிவு அமைப்புகளை உருவாக்குவதற்கான ஒரு நூலகமாகும், இது பாரம்பரிய பழைய அமைப்பான [CLIPS](http://www.clipsrules.net/index.html) போன்றதாக வடிவமைக்கப்பட்டுள்ளது.

நாம் நேரடியாக முன் தொடர் செயல்முறையையும் செயல்படுத்த முடியும், ஆனால் சாதாரண செயலாக்கங்கள் பொதுவாக மிகச் சிறந்த திறன்மிக்கவை அல்ல. பொறுப்பான விதி பொருத்தத்திற்காக ஒரு சிறப்பு அல்காரிதம் [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) பயன்படுத்தப்படுகிறது.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

நாம் எங்கள் முறைமைக்கான வகுப்பை `KnowledgeEngine` என்பதன் துணை வகுப்பாக வரையறுப்போம். ஒவ்வொரு விதிமுறையும் `@Rule` அலங்காரத்துடன் தனித்தனியான פונ்க்ஷன் மூலம் வரையறுக்கப்படுகிறது, இது விதி எப்போது இயங்க வேண்டும் என்பதைக் குறிப்பிடுகிறது. விதியின் உள்ளே, நாம் `declare` פונ்க்ஷன் பயன்படுத்தி புதிய உண்மைகளைச் சேர்க்கலாம், அந்த உண்மைகளைச் சேர்த்தால் முன்னேற்றக் கணத்துப் பொறியால் சில கூடுதல் விதிகள் அழைக்கப்படும்.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

ஒரு அறிவுத்தளத்தை анықித்துவிட்டவுடன், எங்கள் செயற்பாட்டு நினைவகத்தை சில ஆரம்ப உண்மைகளால் நிரப்பி, பிறகு ஊகிக்கையின் செயல்பாட்டை நடத்தியதற்காக `run()` முறைமையைக் கூப்பிடுகிறோம். நாம் முழுமையாக அனைத்து ஆரம்ப உண்மைகளையும் சரியாக அமைத்தால், இறுதியில் அந்த விலங்கின் பற்றிய சரியான உண்மைகள் செயற்பாட்டு நினைவகத்தில் புதிதாக சேர்க்கப்படுவதை நீங்கள் காணலாம்.


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**பொருள் வழங்கல்**:  
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. சரியானதற்காக我们 முயற்சிக்கிறோம், இருப்பினும் தானாக生成ப்பட்ட மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதைக் கவனத்தில் கொள்ளவும். அசல் ஆவணம் அதன் சொந்த மொழியில் அங்கீகாரம் பெறும் ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பின் பயன்பாட்டால் தோன்றும் எந்த தவறானப் புரிதல்கள் அல்லது தவறான விளக்கங்களுக்குமான பொறுப்பும் நாம் ஏற்கவில்லை.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
